In [30]:
import numpy as np
import pandas as pd
import math
from datetime import datetime


#Loading and cleaning data

In [31]:
Tot_Pop = pd.read_csv("./TotalPopulation1960to2020.csv", dtype = str)
CPI = pd.read_csv("./FPCPITOTLZGUSA.csv", dtype=str)
crimes = pd.read_csv("./crimes.csv")
#Tot_Pop.dtypes
#CPI.dtypes
#crimes.dtypes


#Common problem with Pandas getting confused by dates. So I read the dataframe in as a string and did some basic string manipulation. 

In [32]:
Tot_Pop['DATE'] = Tot_Pop['DATE'].str.replace('1/1/', '')
CPI['DATE'] = CPI['DATE'].str.replace('1/1/', '')


In [33]:

def year_convert(two_digit_year):
    if int(two_digit_year) >= 60:
        return 1900 + int(two_digit_year)
    else:
        return 2000 + int(two_digit_year)

# Apply the function to the 'DATE' column
Tot_Pop['DATE'] = Tot_Pop['DATE'].apply(year_convert)
CPI['DATE'] = CPI['DATE'].apply(year_convert)
# Convert 'DATE' back to int
Tot_Pop['DATE'] = Tot_Pop['DATE'].astype(int)
CPI['DATE'] = CPI['DATE'].astype(int)






# Double checking everything is still matching up. 

In [34]:
# # Display the dataframe
#print(Tot_Pop.head())
#print(Tot_Pop.tail())
#print(CPI.head())
#print(CPI.tail())
#CPI.dtypes

    DATE POPTOTUSA647NWDB
57  2017        325122128
58  2018        326838199
59  2019        328329953
60  2020        331501080
61  2021        331893745


In [ ]:
# Renaming columns

In [36]:
crimes = crimes.rename(columns={crimes.columns[10]: 'Larceny theft', crimes.columns[11]: 'Vehicle theft'})
crimes.head()

,Year,Population,Total,Violent,Property,Murder,Rape,Robbery,assault,Burglary,Larceny theft,Vehicle theft
0,1960,"179,323,175",3384200,288460,3095700,9110,17190,107840,154320,912100,1855400,328200
1,1961,"182,992,000",3488000,289390,3198600,8740,17220,106670,156760,949600,1913000,336000
2,1962,"185,771,000",3752200,301510,3450700,8530,17550,110860,164570,994300,2089600,366800
3,1963,"188,483,000",4109500,316970,3792500,8640,17650,116470,174210,1086400,2297800,408300
4,1964,"191,141,000",4564600,364220,4200400,9360,21420,130390,203050,1213200,2514400,472800


# Filtering so we have the same years all the way across. 

In [37]:
crimes = crimes[crimes['Year'] <= 2019]
Tot_Pop = Tot_Pop[Tot_Pop['DATE'] <= 2019]
CPI = CPI[CPI['DATE'] <= 2019]

Making sure everything still lines up. 

In [40]:
#crimes.tail()
#Tot_Pop.tail()
#CPI.tail()

,DATE,FPCPITOTLZGUSA
55,2015,0.118627136
56,2016,1.261583206
57,2017,2.130110004
58,2018,2.442583297
59,2019,1.812210075


# Concatenating the dataframes into one. Cleaning up data type issues. 

In [48]:
dat = pd.concat([CPI, Tot_Pop['POPTOTUSA647NWDB'], crimes.iloc[:, 2:]], axis=1)
dat.columns.values[0:3] = ["Date", "CPI", "Tot_Pop"]
dat['Tot_Pop'] = dat['Tot_Pop'].astype(int)
dat['CPI'] = dat['CPI'].astype(float)
dat.head()

,Date,CPI,Tot_Pop,Total,Violent,Property,Murder,Rape,Robbery,assault,Burglary,Larceny theft,Vehicle theft
0,1960,1.457976,180671000,3384200,288460,3095700,9110,17190,107840,154320,912100,1855400,328200
1,1961,1.070724,183691000,3488000,289390,3198600,8740,17220,106670,156760,949600,1913000,336000
2,1962,1.198773,186538000,3752200,301510,3450700,8530,17550,110860,164570,994300,2089600,366800
3,1963,1.239669,189242000,4109500,316970,3792500,8640,17650,116470,174210,1086400,2297800,408300
4,1964,1.278912,191889000,4564600,364220,4200400,9360,21420,130390,203050,1213200,2514400,472800


# Performing basic calculations on data considering population

In [56]:

dat['crime_percentage'] = dat['Total'] / dat['Tot_Pop'] * 100
dat['CrimeRatePer100k'] = dat['Total'] / dat['Tot_Pop'] * 100000
dat.head()



,Date,CPI,Tot_Pop,Total,Violent,Property,Murder,Rape,Robbery,assault,Burglary,Larceny theft,Vehicle theft,crime_percentage,CrimeRatePer100k
0,1960,1.457976,180671000,3384200,288460,3095700,9110,17190,107840,154320,912100,1855400,328200,1.873129,1873.128504
1,1961,1.070724,183691000,3488000,289390,3198600,8740,17220,106670,156760,949600,1913000,336000,1.898841,1898.840988
2,1962,1.198773,186538000,3752200,301510,3450700,8530,17550,110860,164570,994300,2089600,366800,2.011494,2011.493637
3,1963,1.239669,189242000,4109500,316970,3792500,8640,17650,116470,174210,1086400,2297800,408300,2.171558,2171.558111
4,1964,1.278912,191889000,4564600,364220,4200400,9360,21420,130390,203050,1213200,2514400,472800,2.378771,2378.771060


# Write to csv 

In [ ]:
dat.to_csv('output2.csv', index=False)
